## First Letcure : START -> Generate -> END
## Next : START -> Retrieve -> Generate -> END
### Document : 소득세법
- 우리말 문서는 word document 추천
- 이번 강의에서는 PDF

### 전처리
- 우리말 PDF의 문제 : 줄 바꿈이 일어났을 때 예를들어 "법인세"라는 단어를 -> "법", "인세"로 인식하게 되는 문제가 있다.
  - 임베딩 시 정확도가 떨어지는 문제 따라서 docs를 사용하는 것을 추천
  - 영어는 단어가 바뀔때마다 줄바꿈이됨
- RAG
  - 문서를 통짜로 주면 Token 소모가 매우 큼(고비용) + 시간
  - 필요 없는 문서도 넘어가므로 할루시네이션 문제 증가
  - 문서를 적당한 사이즈로 쪼갠 다음에 LLM에 전달하는 것
- 기본적으로 제공하는 PDF parser는 이미지를 파싱할 수 없다
  - 1. ChatGPT 같은 것 사용해서 전처리 ? -> 일일이 언제 다함 ?
  - 2. zerox -> LLM 활용한 OCR

In [11]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file_path = "./docs/income_tax.pdf"
loader = PyPDFLoader(pdf_file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

/home/luke/Study-2/langchain/kang/.venv/lib/python3.13/site-packages/pypdf/generic/_base.py:892: RuntimeWarning: coroutine 'main' was never awaited
  return NameObject(ret)


CancelledError: 

In [ ]:
pages[35] # 원하는 내용이 55조 세율 내용이라면 인덱싱 가능

Document(metadata={'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2025-08-05T15:03:16+09:00', 'moddate': '2025-08-05T15:03:16+09:00', 'source': './docs/income_tax.pdf', 'total_pages': 130, 'page': 35, 'page_label': '36'}, page_content='법제처                                                            36                                                       국가법령정보센터\n소득세법\n[본조신설 2022. 12. 31.]\n \n제58조(재해손실세액공제) ① 사업자가 해당 과세기간에 천재지변이나 그 밖의 재해(이하 “재해”라 한다)로 대통령령으\n로 정하는 자산총액(이하 이 항에서 “자산총액”이라 한다)의 100분의 20 이상에 해당하는 자산을 상실하여 납세가\n곤란하다고 인정되는 경우에는 다음 각 호의 소득세액(사업소득에 대한 소득세액을 말한다. 이하 이 조에서 같다)에\n그 상실된 가액이 상실 전의 자산총액에서 차지하는 비율(이하 이 조에서 “자산상실비율”이라 한다)을 곱하여 계산\n한 금액(상실된 자산의 가액을 한도로 한다)을 그 세액에서 공제한다. 이 경우 자산의 가액에는 토지의 가액을 포함\n하지 아니한다. <개정 2020. 12. 29.>\n1. 재해 발생일 현재 부과되지 아니한 소득세와 부과된 소득세로서 미납된 소득세액\n2. 재해 발생일이 속하는 과세기간의 소득에 대한 소득세액\n② 제1항의 경우에 제56조, 제56조의2, 제57조 및 제57조의2에 따라 공제할 세액이 있을 때에는 이를 공제한 후의\n세액을 소득세액으로 하여 제1항을 적용한다.<개정 2022. 12. 31.>\n③ 제1항에 따른 공제를 “재해손실세액공제”라 한다

In [ ]:
import nest_asyncio
nest_asyncio.apply() # Jupyter 에서 asyncio를 사용할 때 (pyzerox는 event loop가 있으면 작동하지 않음)

In [ ]:
from pyzerox import zerox
import asyncio

### 모델 설정 (Vision 모델만 사용) 참고: https://docs.litellm.ai/docs/providers ###

## 일부 모델에 필요할 수 있는 추가 모델 kwargs의 자리 표시자
kwargs = {}

## Vision 모델에 사용할 시스템 프롬프트
custom_system_prompt = None

model = "gpt-4o-mini" 

# 메인 비동기 진입점을 정의합니다
async def main():
    file_path = "./docs/income_tax.pdf" ## 로컬 파일 경로 및 파일 URL 지원

    ## 일부 페이지 또는 전체 페이지를 처리
    select_pages = None ## 전체는 None, 특정 페이지는 int 또는 list(int) 페이지 번호 (1부터 시작)

    output_dir = "./docs" ## 통합된 마크다운 파일을 저장할 디렉토리
    result = await zerox(file_path=file_path, model=model, output_dir=output_dir,
                        custom_system_prompt=custom_system_prompt, select_pages=select_pages, **kwargs)
    return result


# 메인 함수를 실행합니다:
result = asyncio.run(main())

# 마크다운 결과를 출력합니다
print(result)

ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-N9GDqxsHYOw91R7wSv352Bcw on tokens per min (TPM): Limit 60000, Used 60000, Requested 807. Please try again in 807ms. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.
    Please check the status of your model provider API status.
    



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-N9GDqxsHYOw91R7wSv352Bcw on tokens per min (TPM): Limit 60000, Used 60000, Requested 807. Please try again in 807ms. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.
    Please check the status of your model provider API status.
    



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-N9GDqxsHYOw91R7wSv352Bcw on tokens per min (TPM): Limit 60000, Used 60000, Requested 807. Please try again in 807ms. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.
    Please check the status of your model provider API status.
    
ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-N9GDqxsHYOw91R7wSv352Bcw on tokens per min (TPM): Limit 60000, Used 60000, Requested 807. Please try again in 807ms. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a

...

ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-N9GDqxsHYOw91R7wSv352Bcw on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.
    Please check the status of your model provider API status.
    



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-N9GDqxsHYOw91R7wSv352Bcw on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.
    Please check the status of your model provider API status.
    



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



KeyboardInterrupt: 

# pyzerox
- 문서 파싱 결과는 MarkDown
  - 표도 마크다운으로 정리됨
  - 이 결과 MarkDown을 로딩하면됨
- 사용 패키지 : "unstructured[md]" nltk -> 같이 쓰는 것을 langchain에서 권장
- PDF에서는 문서를 page 단위로 쪼갰었음
- Markdown은 split을 해줘야함

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100,
    separators=['\n\n', '\n']
)

In [25]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

markdown_path = "./docs/income_tax.md"
loader = UnstructuredMarkdownLoader(markdown_path)
document_list = loader.load_and_split(text_splitter)

In [27]:
document_list[40]

Document(metadata={'source': './docs/income_tax.md'}, page_content='제2절 세액의 계산 <개정 2009. 12. 31.> 제1관 세율 <개정 2009. 12. 31.>\n\n제55조(세율) 거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 "종합소득과세표준세액"이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 ┌───────────────┐ │ 과세표준의 6개 구간 │ ├───────────────┤ │ 1,400만원 이하 │ 84만원 + (1,400만원을 초과하는 금액의 15%) │ │ 1,400만원 초과 │ 84만원 + (5,000만원을 초과하는 금액의 24%) │ │ 8,800만원 이하 │ 624만원 + (5,000만원을 초과하는 금액의 24%) │ │ 8,800만원 초과 │ 1,536만원 + (8,800만원을 초과하는 금액의 35%) │ │ 1.5억원 초과 │ 4,046만원 + (1,500만원을 초과하는 금액의 38%) │ │ 3억원 초과 │ 6,460만원 + (3억원을 초과하는 금액의 40%) │ │ 5억원 초과 │ 14,760만원 + (5억원을 초과하는 금액의 42%) │ │ 10억원 초과 │ 38,406만원 + (10억원을 초과하는 금액의 45%) │ └───────────────┘\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 ‘퇴직소득 산출세액’이라 한다)으로 한다. <개정 2013. 1. 1., 2014. 12. 23.> 1. 해당 과세기간의 퇴직소득과세표준 제1항의 세율을 적용하여 계산한 금액 2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액 3. 삭제<2014. 12. 23.> [전문개정 2009. 12. 31.]\n\n제2관 세액공제 <개정 2

- markdown의 전처리 결과가 이전보다는 낫지만 표의 내용이 짤리게된다. -> 정확한 Context를 전달하게 되는 것은 아니게됨
- md -> text -> load -> split 과정이 필요해짐
  - 추가 설치 패키지 : html2text beautifulsoup4

In [17]:
import markdown
from bs4 import BeautifulSoup

text_path = './docs/income_tax.txt'

# 마크다운 파일을 읽어옵니다
with open(markdown_path, 'r', encoding='utf-8') as md_file:
    md_content = md_file.read()

# 마크다운 콘텐츠를 HTML로 변환합니다
html_content = markdown.markdown(md_content)

# HTML 콘텐츠를 파싱하여 텍스트만 추출합니다
soup = BeautifulSoup(html_content, 'html.parser')
text_content = soup.get_text()

# 추출한 텍스트를 텍스트 파일로 저장합니다
with open(text_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write(text_content)

print("Markdown converted to plain text successfully!")

Markdown converted to plain text successfully!


In [20]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('./docs/income_tax.txt')
document_list = loader.load_and_split(text_splitter)

In [24]:
document_list[47]

Document(metadata={'source': './docs/income_tax.txt'}, page_content='② 제70조제1항, 제70조의2에 따른 제74조에 따라 차례로 할 것이 제70조제1항제2호에 따르며 서류를 제출하여야 한다는 경우에는 기준소득 중 거주자 본인이 된다(분산)과 제70조제2와 제74조에 따른 제료 및 제대법을 포함한다. 단, 차별제표청정인 그 업체를 남겨 제출한 경우로 그에 대하여 아니하다.<개정 2013. 1. 1.>\n  ③ 제80조에 따른 수익과 관련의 경우에는 기초공제 중 거주자 본인이 된다(분산)과 그에 관한 적지사항을 분명히 한다.\n[전문개정 2009. 12. 31.]\n[제목개정 2014. 1. 1.]\n제54조의2(공동사업에 대한 소득공제 등 특례) 제51조의3 또는 「조세특례제한법」에 따른 소득공제를 적용하거나 제59조의2에 따른 세액감면을 적용하는 경우 제54조제3항에 따라 공동사업자의 소득에 합산과세되는 특별세액거래의 지출․납입․투자 등의 금액이 있을 경우 주된 공동사업자의 소득에 합산과세되는 소득금액에 합산되어 주된 공동사업자의 합산과세세액은 공동사업소득액 또는 공동사업창출세액을 계산할 때 소득공제 또는 세액공제를 받을 수 있다. \n[개정 2014. 1. 1.]\n[전문개정 2009. 12. 31.]\n[제목개정 2014. 1. 1.]\n제2절 세액의 계산 <개정 2009. 12. 31.>\n제1관 세율 <개정 2009. 12. 31.>\n제55조(세율) 거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 "종합소득과세표준세액"이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n종합소득\n┌───────────────┐\n│ 과세표준의 6개 구간 │\n├───────────────┤\n│ 1,400만원 이하        │ 84

In [28]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(documents=document_list,
                                     embedding=embeddings,
                                     collection_name="income_tax_collection",
                                     persist_directory="./income_tax_collection")
    

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [30]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

NameError: name 'vector_store' is not defined

In [31]:
query = "연봉 5천만원 직장인의 소득세는 ?"

In [32]:
retriever.invoke(query)

NameError: name 'retriever' is not defined

In [33]:
from typing_extensions import TypedDict
from langchain_core.documents import Document

class AgentState(TypedDict):
    query: str
    context: list[Document]
    answer: str

In [34]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

In [35]:
def retrieve(state: AgentState):
    query = state["query"]
    docs = retriever.invoke(query)
    
    return {"context": docs}

In [ ]:
from langsmith import Client
import os
def genereate(state: AgentState):
    client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))
    prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)